In [1]:
import os
import mat_properties as prop
import numpy as n
import pandas as pd
import cotel
import nasos
import time
from scipy.optimize import root

#Время начала расчёта
start_time = time.time()

#Основные константы
calctolerance=10**-5
calcmethod = "hybr"
KPDnasos = 0.9
KPD=0.99

#таблица ном режима
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
#таблица задаваемых значений (заменится на интерфейс)
gas_streamsZAD    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streamsZAD  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)
#рабочая таблица (=номинал в 1 итерации)
gas_streams = gas_streams0.copy()
water_streams = water_streams0.copy()


#чтение с вводного экселя входных данных
Tgas = gas_streamsZAD.loc['GTU-PEVD','T']
Pgas = gas_streamsZAD.loc['GTU-PEVD','P']
Ggas = gas_streamsZAD.loc['GTU-PEVD','G']
Propgas = gas_streamsZAD.loc['GTU-PEVD','N2':'Ar']
Pvd = water_streamsZAD.loc['IVD-PEVD','P']
Pnd = water_streamsZAD.loc['IND-PPND','P']
Twat0 = water_streamsZAD.loc['X-GPK','T']

#запись состава газов в раб табл
gas_streams['N2'] = Propgas[0]
gas_streams['O2'] = Propgas[1]
gas_streams['CO2'] = Propgas[2]
gas_streams['H2O'] = Propgas[3]
gas_streams['Ar'] = Propgas[4]


#Состав газов в номинальном режиме и в частичной нагрузке
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
fractiongas0 = (gas_streams0.at['GTU-PEVD','N2'],gas_streams0.at['GTU-PEVD','O2'],gas_streams0.at['GTU-PEVD','CO2'],gas_streams0.at['GTU-PEVD','H2O'],gas_streams0.at['GTU-PEVD','Ar'])
fractiongas  = (gas_streams.at['GTU-PEVD','N2'],gas_streams.at['GTU-PEVD','O2'],gas_streams.at['GTU-PEVD','CO2'],gas_streams.at['GTU-PEVD','H2O'],gas_streams.at['GTU-PEVD','Ar'])

#Считывание рефпропа
RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'
gas1 = prop.Materials_prop(gasmix,fractiongas, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
gas0 = prop.Materials_prop(gasmix,fractiongas0, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)

#Задание в раб табл параметров газа (1 строка)
gas_streams.loc['GTU-PEVD','T'] = Tgas
gas_streams['G'] = Ggas
gas_streams['P'] = Pgas
gas_streams.loc['GTU-PEVD','H'] = gas1.p_t(Pgas,Tgas)['h']

#Задание в раб табл параметров по воде
water_streams.loc['IVD-PEVD':'PEN-EVD','P'] = Pvd
water_streams.loc['IND-PPND':'X-GPK','P'] = Pnd
water_streams.loc['BND-PEN','P'] = Pnd
water_streams.loc['X-GPK','T'] = Twat0

#Расчёт параметров на насыщении в испарительных поверхностях (недогрев до нас на входе 3°С)
water_streams.loc['IVD-PEVD','T'] = water.p_q(Pvd,1)['T']
water_streams.loc['IVD-PEVD','H'] = water.p_q(Pvd,1)['h']
water_streams.loc['IND-PPND','T'] = water.p_q(Pnd,1)['T']
water_streams.loc['IND-PPND','H'] = water.p_q(Pnd,1)['h']
water_streams.loc['EVD-IVD','T'] = water.p_q(Pvd,1)['T']-3
water_streams.loc['GPK-IND','T'] = water.p_q(Pnd,1)['T']-3
water_streams.loc['EVD-IVD','H'] = water.p_t(Pvd,water_streams.loc['EVD-IVD','T'])['h']
water_streams.loc['GPK-IND','H'] = water.p_t(Pnd,water_streams.loc['GPK-IND','T'])['h']


it=50
for k in range(it):
    ######################################################################
    #Связвка высокого давления
    for j in range(it):
        for i in range(it):
            PEVD = cotel.heatex('GTU-PEVD','PEVD-IVD','IVD-PEVD','PEVD-DROS',       KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['PEVD-IVD','T':'G'] = [PEVD[0],PEVD[1],PEVD[2],PEVD[3]]
            water_streams.loc['PEVD-DROS','T':'G'] = [PEVD[4],PEVD[5],PEVD[6],PEVD[7]]
            IVD  = cotel.vapor ('PEVD-IVD','IVD-EVD','EVD-IVD','IVD-PEVD',           KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['IVD-EVD','T':'G'] = [IVD[0],IVD[1],IVD[2],IVD[3]]
            water_streams.loc['IVD-PEVD','T':'G'] = [IVD[4],IVD[5],IVD[6],IVD[7]]
            water_streams.loc['PEVD-DROS':'PEN-EVD','G'] = IVD[7]
            water_streams.loc['BND-PEN','G'] = IVD[7]
            Qgas = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['PEVD-IVD','H'])
            Qwat = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['IVD-PEVD','H'])
            print('dQ/Q ПЕВД+ИВД',((Qgas-Qwat)/Qgas))
            if  abs((Qgas-Qwat)/Qgas) < calctolerance:
                break;
        EVD = cotel.heatex('IVD-EVD','EVD-PPND','PEN-EVD','EVD-IVD',               KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['EVD-PPND','T':'G'] = [EVD[0],EVD[1],EVD[2],EVD[3]]
        water_streams.loc['EVD-IVD','T':'G'] = [EVD[4],EVD[5],EVD[6],EVD[7]]

        Qgas1 = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
        Qwat1 = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['EVD-IVD','H'])
        print('dQ/Q ПЕВД+ИВД+ЭВД',((Qgas1-Qwat1)/Qgas1))
        if  abs((Qgas1-Qwat1)/Qgas1) < calctolerance:
            break;
    print("ВД+ --- %s сек. ---" % round((time.time() - start_time)))
    ######################################################################

# #от отрицательной Q
# gas_streams.loc['PPND-IND','T'] = gas_streams.loc['EVD-PPND','T']-10
# gas_streams.loc['PPND-IND','H'] = gas1.p_t(gas_streams.loc['PPND-IND','P'],gas_streams.loc['PPND-IND','T'])['h']



    for j in range(it):
        #Связка ППНД+ИНД
        for i in range(it):
            #Расчёт ППНД
            PPND = cotel.heatex('EVD-PPND','PPND-IND','IND-PPND','PPND-TURB',    KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['PPND-IND','T':'G']    = [PPND[0],PPND[1],PPND[2],PPND[3]]
            water_streams.loc['PPND-TURB','T':'G'] = [PPND[4],PPND[5],PPND[6],PPND[7]]
            
            #Расчёт ИНД
            IND  = cotel.vaporND('PPND-IND','IND-GPK','GPK-IND','IND-PPND',      KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['IND-GPK','T':'G']    = [IND[0],IND[1],IND[2],IND[3]]
            water_streams.loc['IND-PPND','T':'G'] = [IND[4],IND[5],IND[6],IND[7]]
            #Переопределение расхода пара
            water_streams.loc['PPND-TURB':'IND-PPND','G'] = IND[7]
            
            
            #ПЭН
            water_streams.loc['BND-PEN','T':'G'] = [IND[9],IND[10],IND[11],IND[12]]
            PEN=nasos.nasos('BND-PEN','PEN-EVD',water,KPDnasos,water_streams).calc()
            water_streams.loc['PEN-EVD','T':'G'] = [PEN[0],PEN[1],PEN[2],PEN[3]]
            
            Qgas = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['IND-GPK','H'])
            Qwat = water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
            print('dQ/Q ППНД+ИНД',((Qgas-Qwat)/Qgas))
            if  abs((Qgas-Qwat)/Qgas) < calctolerance:
                break;

        #Расчёт расхода в ГПК
        water_streams.loc['X-GPK','G'] = water_streams.at['PPND-TURB','G']+water_streams.at['PEVD-DROS','G']
        water_streams.loc['GPK-IND','G'] = water_streams.at['X-GPK','G']
        #Расчёт ГПК
        GPK = cotel.heatex('IND-GPK','GPK-out','X-GPK','GPK-IND',           KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['GPK-out','T':'G']   = [GPK[0],GPK[1],GPK[2],GPK[3]]
        water_streams.loc['GPK-IND','T':'G'] = [GPK[4],GPK[5],GPK[6],GPK[7]]

        Qgas1 = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['GPK-out','H'])
        Qwat1 = water_streams.at['GPK-IND','G']*(water_streams.at['GPK-IND','H']-water_streams.at['X-GPK','H'])+water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
        print('dQ/Q ППНД+ИНД+ГПК',(Qgas1-Qwat1)/Qgas1)
        if  abs((Qgas1-Qwat1)/Qgas1) < calctolerance:
            break;
    print("НД+ --- %s сек. ---" % round((time.time() - start_time)))
    ######################################################################
    
    
    
    Qgasall = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['GPK-out','H'])
    Qwatall = water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['X-GPK','H'])+water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['X-GPK','H'])-water_streams.at['BND-PEN','G']*(water_streams.at['PEN-EVD','H']-water_streams.at['BND-PEN','H'])
    print('dQ/Qsumm',(Qgasall-Qwatall)/Qgasall)
    if  abs((Qgasall-Qwatall)/Qgasall) < calctolerance:
        break;

print('Степень сухости пара в ЭВД: ',water.p_h(water_streams.at['EVD-IVD','P'],water_streams.at['EVD-IVD','H'])['Q'])
print('Степень сухости пара в ГПК: ',water.p_h(water_streams.at['GPK-IND','P'],water_streams.at['GPK-IND','H'])['Q'])
print("fin:--- %s сек. ---" % round((time.time() - start_time)))

dQ/Q ПЕВД+ИВД 0.07969935038847616
dQ/Q ПЕВД+ИВД -0.02677672533515567
dQ/Q ПЕВД+ИВД 0.008273545653333802
dQ/Q ПЕВД+ИВД -0.0026259278544222835
dQ/Q ПЕВД+ИВД 0.0008264512484871365
dQ/Q ПЕВД+ИВД -0.00026079951126198434
dQ/Q ПЕВД+ИВД 8.223036612527001e-05
dQ/Q ПЕВД+ИВД -2.5934179498679056e-05
dQ/Q ПЕВД+ИВД 8.178555172004956e-06
dQ/Q ПЕВД+ИВД+ЭВД 0.014441208692110708
dQ/Q ПЕВД+ИВД -0.00539171210378122
dQ/Q ПЕВД+ИВД 0.001722872133817164
dQ/Q ПЕВД+ИВД -0.0005535577181032886
dQ/Q ПЕВД+ИВД 0.00017754513411633854
dQ/Q ПЕВД+ИВД -5.6977037760876904e-05
dQ/Q ПЕВД+ИВД 1.8281519096087637e-05
dQ/Q ПЕВД+ИВД -5.866106465925725e-06
dQ/Q ПЕВД+ИВД+ЭВД 0.01956116972412354
dQ/Q ПЕВД+ИВД -0.029306449583838184
dQ/Q ПЕВД+ИВД 0.009282289644047037
dQ/Q ПЕВД+ИВД -0.0030276875396390264
dQ/Q ПЕВД+ИВД 0.0009782745044372967
dQ/Q ПЕВД+ИВД -0.0003170622349294282
dQ/Q ПЕВД+ИВД 0.00010265887145061485
dQ/Q ПЕВД+ИВД -3.324974565950234e-05
dQ/Q ПЕВД+ИВД 1.0767995677837248e-05
dQ/Q ПЕВД+ИВД -3.487354418464431e-06
dQ/Q ПЕВД+ИВД

In [2]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
GTU-PEVD,522.800000,0.1,936.681566,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
PEVD-IVD,456.193861,0.1,860.856287,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
IVD-EVD,308.685068,0.1,697.104135,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
EVD-PPND,235.099362,0.1,617.476225,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
PPND-IND,232.099319,0.1,614.256343,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
IND-GPK,175.323998,0.1,553.673338,516.2,0.7803,0.1237,0.0301,0.0594,0.0065
GPK-out,107.193963,0.1,481.741286,516.2,0.7803,0.1237,0.0301,0.0594,0.0065


In [3]:
water_streams

,T,P,H,G
PEVD-DROS,496.985531,8.012649,3391.866880,60.701314
IVD-PEVD,297.938037,8.341200,2753.507839,60.701314
EVD-IVD,297.938037,8.341200,1374.881153,60.701314
PEN-EVD,165.671153,8.341200,704.502121,60.701314
BND-PEN,164.512241,0.692500,695.109778,60.701314
PPND-TURB,207.878498,0.685916,2863.528067,16.254389
IND-PPND,164.512241,0.692500,2762.297432,16.254389
GPK-IND,164.512241,0.692500,729.425953,76.955703
X-GPK,60.000000,0.692500,251.748396,76.955703


In [4]:
Qgas = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['PEVD-IVD','H'])
Qwat = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['IVD-PEVD','H'])
print('dQ/Q ПЕВД+ИВД',((Qgas-Qwat)/Qgas))

Qgas1 = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
Qwat1 = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['EVD-IVD','H'])
print('dQ/Q ПЕВД+ИВД+ЭВД',((Qgas1-Qwat1)/Qgas1))

Qgas = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['IND-GPK','H'])
Qwat = water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
print('dQ/Q ППНД+ИНД',((Qgas-Qwat)/Qgas))

Qgas1 = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['GPK-out','H'])
Qwat1 = water_streams.at['GPK-IND','G']*(water_streams.at['GPK-IND','H']-water_streams.at['X-GPK','H'])+water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
print('dQ/Q ППНД+ИНД+ГПК',(Qgas1-Qwat1)/Qgas1)

Qgasall = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['GPK-out','H'])
Qwatall = water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['X-GPK','H'])+water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['X-GPK','H'])-water_streams.at['BND-PEN','G']*(water_streams.at['PEN-EVD','H']-water_streams.at['BND-PEN','H'])
print('dQ/Qsumm',(Qgasall-Qwatall)/Qgasall)

dQ/Q ПЕВД+ИВД 9.457479828067239e-06
dQ/Q ПЕВД+ИВД+ЭВД -4.185962528157558e-06
dQ/Q ППНД+ИНД 6.475405797944894e-06
dQ/Q ППНД+ИНД+ГПК 3.043796880029906e-06
dQ/Qsumm -1.2962417036833713e-06
